In [ ]:
import logging
import functools
import json
import time
from abc import ABC, abstractmethod
from enum import Enum
from typing import Dict, List, Optional, Union, Any
from confluent_kafka import Producer, Consumer
from confluent_kafka.serialization import StringSerializer
import requests


class DataStoreType(Enum):
    """Enumeration of supported data store types"""
    KAFKA = "kafka"
    ADLS = "adls"


class ApplicationConfig:
    """
    Central application configuration that manages environment-specific configurations
    """
    
    def __init__(self):
        self._config_dict = {
            "nonprod": {
                # Kafka Configuration
                "kafka": {
                    "bootstrap-servers": "nonprod-kafka-broker1:9092,nonprod-kafka-broker2:9092",
                    "client-id-key": "nonprod-kafka-client-id",
                    "client-secret-key": "nonprod-kafka-client-secret",
                    "token-url": "https://nonprod-auth.company.com/oauth/token",
                    "ssl-ca-location": "kafka-nonprod.pem",
                    "default-group-id": "nonprod-insurance-data-eng"
                },
                # ADLS Configuration
                "adls": {
                    "client-id-key": "nonprod-adls-client-id",
                    "client-secret-key": "nonprod-adls-client-secret",
                    "tenant-id-key": "nonprod-tenant-id"  # Changed to key reference
                },
                # Common Configuration
                "default-secret-scope": "nonprod-secrets"
            },
            "prod": {
                # Kafka Configuration
                "kafka": {
                    "bootstrap-servers": "prod-kafka-broker1:9092,prod-kafka-broker2:9092",
                    "client-id-key": "prod-kafka-client-id",
                    "client-secret-key": "prod-kafka-client-secret",
                    "token-url": "https://prod-auth.company.com/oauth/token",
                    "ssl-ca-location": "kafka-prod.pem",
                    "default-group-id": "prod-insurance-data-eng"
                },
                # ADLS Configuration
                "adls": {
                    "client-id-key": "prod-adls-client-id", 
                    "client-secret-key": "prod-adls-client-secret",
                    "tenant-id-key": "prod-tenant-id"  # Changed to key reference
                },
                # Common Configuration
                "default-secret-scope": "prod-secrets"
            }
        }
    
    def get_config_dict(self) -> Dict[str, Any]:
        """Get the complete configuration dictionary"""
        return self._config_dict.copy()
    
    def get_configuration(self, env: str, service_type: str) -> Dict[str, Any]:
        """
        Get configuration for a specific environment and service type
        
        Args:
            env (str): Environment name ('prod' or 'nonprod')
            service_type (str): Service type ('kafka', 'adls', etc.)
            
        Returns:
            dict: Configuration dictionary for the service
        """
        if env not in self._config_dict:
            raise ValueError(f"Environment '{env}' not found in configurations")
        
        env_config = self._config_dict[env]
        
        if service_type not in env_config:
            raise ValueError(f"Service type '{service_type}' not found for environment '{env}'")
        
        # Merge service-specific config with common config
        config = env_config[service_type].copy()
        config['default-secret-scope'] = env_config['default-secret-scope']
        
        return config
    
    def add_environment_config(self, env: str, service_type: str, config_dict: Dict[str, Any]):
        """
        Add or update configuration for an environment and service type
        
        Args:
            env (str): Environment name
            service_type (str): Service type
            config_dict (dict): Configuration dictionary
        """
        if env not in self._config_dict:
            self._config_dict[env] = {}
        
        self._config_dict[env][service_type] = config_dict


class ConfigurationService:
    """
    Configuration service that provides a unified interface for accessing configurations
    and resolving secrets from Databricks
    """
    
    def __init__(self, application_config: Optional[ApplicationConfig] = None):
        self.application_config = application_config or ApplicationConfig()
        self.logger = logging.getLogger(__name__)
    
    def get_configuration(self, env: str, service_type: str = None) -> Dict[str, Any]:
        """
        Get configuration for environment and optional service type
        
        Args:
            env (str): Environment name
            service_type (str): Optional service type filter
            
        Returns:
            dict: Configuration dictionary
        """
        if service_type:
            return self.application_config.get_configuration(env, service_type)
        else:
            # Return all configurations for the environment
            return self.application_config.get_config_dict()[env]
    
    def get_resolved_configuration(self, env: str, service_type: str, 
                                 custom_secret_scope: Optional[str] = None) -> Dict[str, Any]:
        """
        Get configuration with secrets resolved from Databricks
        
        Args:
            env (str): Environment name
            service_type (str): Service type
            custom_secret_scope (str): Optional custom secret scope
            
        Returns:
            dict: Configuration with resolved secret values
        """
        config = self.get_configuration(env, service_type)
        secret_scope = custom_secret_scope or config.get('default-secret-scope')
        
        resolved_config = config.copy()
        
        # Resolve secrets for keys that end with '-key'
        for key, value in config.items():
            if key.endswith('-key'):
                try:
                    resolved_value = self._get_secret(secret_scope, value)
                    # Store resolved value with new key name (remove '-key' suffix)
                    resolved_key = key.replace('-key', '')
                    resolved_config[resolved_key] = resolved_value
                    self.logger.info(f"Resolved secret for {key}")
                except Exception as e:
                    self.logger.error(f"Failed to resolve secret for {key}: {str(e)}")
                    raise Exception(f"Failed to resolve secret for {key}: {str(e)}")
        
        return resolved_config
    
    def _get_secret(self, scope: str, key: str) -> str:
        """
        Get secret value from Databricks secret scope
        
        Args:
            scope (str): Secret scope name
            key (str): Secret key name
            
        Returns:
            str: Secret value
        """
        try:
            # In a real Databricks environment, this would be:
            # return dbutils.secrets.get(scope, key)
            
            # For demonstration/testing purposes:
            return f"resolved_secret_from_{scope}_{key}"
            
        except Exception as e:
            self.logger.error(f"Failed to retrieve secret '{key}' from scope '{scope}': {str(e)}")
            raise


class DataStoreConnectionFactory:
    """
    Factory class for creating data store connections based on type
    """
    
    @staticmethod
    def create_connection(datastore_type: DataStoreType, env: str, 
                         config_service: Optional[ConfigurationService] = None,
                         **kwargs):
        """
        Create a connection based on the datastore type
        
        Args:
            datastore_type (DataStoreType): Type of datastore to connect to
            env (str): Environment name
            config_service (ConfigurationService): Configuration service instance
            **kwargs: Additional arguments for specific connectors
            
        Returns:
            Connection object based on the datastore type
        """
        config_service = config_service or ConfigurationService()
        
        if datastore_type == DataStoreType.ADLS:
            return ADLSConnector(env=env, config_service=config_service, **kwargs)
        elif datastore_type == DataStoreType.KAFKA:
            return KafkaProducer(env=env, config_service=config_service, **kwargs)
        else:
            raise ValueError(f"Unsupported datastore type: {datastore_type}")


class ADLSConnector:
    """
    Azure Data Lake Storage connector implementation
    """
    
    def __init__(self, env: str = 'nonprod', config_service: Optional[ConfigurationService] = None, 
                 safe_parent_path: str = "/mnt/temp", custom_secret_scope: Optional[str] = None):
        """
        Initialize the ADLS connector
        
        Args:
            env (str): Environment ('prod' or 'nonprod')
            config_service (ConfigurationService): Configuration service instance
            safe_parent_path (str): Safe parent path for operations
            custom_secret_scope (str): Optional custom secret scope
        """
        self.env = env
        self.config_service = config_service or ConfigurationService()
        self.safe_parent_path = safe_parent_path
        self.custom_secret_scope = custom_secret_scope
        self.logger = logging.getLogger(__name__)
        
        # Get resolved configuration (secrets resolved by ConfigurationService)
        self.env_config = self.config_service.get_resolved_configuration(
            env, 'adls', custom_secret_scope
        )
        
        self.config_dict = None
        self.source_path = None
        self.mount_point = None
        self.is_mounted = False
        
    def setup_connection(self, storage_account_name: str, container_name: str, 
                        mount_name: str, subfolder_path: str = ""):
        """
        Set up the ADLS connection configuration
        
        Args:
            storage_account_name (str): Azure storage account name
            container_name (str): Container name
            mount_name (str): Local mount name
            subfolder_path (str): Optional subfolder path
        """
        # Use resolved secrets from configuration service
        client_id = self.env_config['client-id']
        client_secret = self.env_config['client-secret']
        tenant_id = self.env_config['tenant-id']
        
        # Build Azure storage path
        self.source_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{subfolder_path}"
        self.mount_point = f"/mnt/{mount_name}"
        
        # Create OAuth configuration
        self.config_dict = {
            "fs.azure.account.auth.type": "OAuth",
            "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
            "fs.azure.account.oauth2.client.id": client_id,
            "fs.azure.account.oauth2.client.secret": client_secret,
            "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
        }
        
        self.logger.info(f"ADLS configuration set up for {self.env} environment")
        self.logger.info(f"Source: {self.source_path}, Mount: {self.mount_point}")
    
    def mount(self):
        """Mount the ADLS storage to Databricks file system"""
        if not self.config_dict:
            raise Exception("Configuration not set up. Call setup_connection() first.")
        
        try:
            # In real Databricks environment:
            # dbutils.fs.mount(
            #     source=self.source_path,
            #     mount_point=self.mount_point,
            #     extra_configs=self.config_dict
            # )
            self.is_mounted = True
            self.logger.info(f"Successfully mounted {self.source_path} at {self.mount_point}")
            
        except Exception as e:
            if "already mounted" in str(e).lower():
                self.is_mounted = True
                self.logger.info(f"{self.mount_point} already mounted")
            else:
                self.logger.error(f"Failed to mount ADLS: {str(e)}")
                raise Exception(f"Failed to mount ADLS: {str(e)}")
    
    def unmount(self):
        """Unmount the ADLS storage"""
        if not self.mount_point:
            raise Exception("No mount point configured")
            
        try:
            # In real Databricks environment:
            # dbutils.fs.unmount(self.mount_point)
            self.is_mounted = False
            self.logger.info(f"Successfully unmounted {self.mount_point}")
            
        except Exception as e:
            self.logger.error(f"Could not unmount {self.mount_point}: {str(e)}")
            raise


class KafkaProducer:
    """
    Kafka producer for publishing messages to topics
    """
    
    def __init__(self, env: str = 'nonprod', topic_name: Optional[str] = None, 
                 config_service: Optional[ConfigurationService] = None, 
                 custom_secret_scope: Optional[str] = None):
        """
        Initialize the Kafka producer
        
        Args:
            env (str): Environment ('prod' or 'nonprod')
            topic_name (str): Default topic name
            config_service (ConfigurationService): Configuration service instance
            custom_secret_scope (str): Custom secret scope
        """
        self.env = env
        self.topic_name = topic_name
        self.config_service = config_service or ConfigurationService()
        self.custom_secret_scope = custom_secret_scope
        self.logger = logging.getLogger(__name__)
        
        # Get resolved configuration (secrets resolved by ConfigurationService)
        self.env_config = self.config_service.get_resolved_configuration(
            env, 'kafka', custom_secret_scope
        )
        
        # Setup credentials from resolved config
        self.client_id = self.env_config['client-id']
        self.client_secret = self.env_config['client-secret']
        self.bootstrap_servers = self.env_config['bootstrap-servers']
        self.token_url = self.env_config['token-url']
        self.ssl_ca_location = self.env_config['ssl-ca-location']
        
        # Create producer
        self.producer = Producer(self._get_producer_config())
        self.serializer = StringSerializer('utf8')
        
        self.logger.info(f"Kafka Producer initialized for {self.env} environment")
        self.logger.info(f"Bootstrap servers: {self.bootstrap_servers}")
    
    def _get_token(self, config):
        """Get OAuth token for authentication"""
        payload = {
            'grant_type': 'client_credentials',
            'scope': 'kafka'
        }
        
        resp = requests.post(
            self.token_url,
            auth=(self.client_id, self.client_secret),
            data=payload
        )
        
        token = resp.json()
        return token['access_token'], time.time() + float(token['expires_in'])
    
    def _get_producer_config(self) -> Dict[str, Any]:
        """Get producer configuration"""
        return {
            'bootstrap.servers': self.bootstrap_servers,
            'security.protocol': 'sasl_ssl',
            'sasl.mechanisms': 'OAUTHBEARER',
            'ssl.ca.location': self.ssl_ca_location,
            'oauth_cb': functools.partial(self._get_token),
            'logger': self.logger,
        }
    
    def send_message(self, message: Union[Dict, str], topic_name: Optional[str] = None) -> bool:
        """
        Send a message to the specified topic
        
        Args:
            message (dict or str): Message to send
            topic_name (str): Topic to send to
            
        Returns:
            bool: Success status
        """
        if not topic_name and not self.topic_name:
            raise Exception("No topic specified. Provide topic_name parameter or set default topic.")
            
        target_topic = topic_name or self.topic_name
        
        # Convert message to JSON string if it's a dict
        if isinstance(message, dict):
            message = json.dumps(message)
        
        try:
            self.producer.produce(target_topic, value=message)
            self.producer.flush()
            self.logger.info(f"Message sent to topic '{target_topic}'")
            return True
        except Exception as e:
            self.logger.error(f"Failed to send message: {e}")
            return False
    
    def close(self):
        """Close the producer connection"""
        self.producer.flush()
        self.logger.info("Producer closed")


# Usage Examples:
if __name__ == "__main__":
    # Example 1: Using the factory pattern
    config_service = ConfigurationService()
    
    # Create ADLS connection using factory
    adls_conn = DataStoreConnectionFactory.create_connection(
        DataStoreType.ADLS, 
        env='nonprod',
        config_service=config_service
    )
    
    # Create Kafka connection using factory
    kafka_conn = DataStoreConnectionFactory.create_connection(
        DataStoreType.KAFKA,
        env='nonprod', 
        config_service=config_service,
        topic_name='my-topic'
    )
    
    # Example 2: Direct instantiation (still works)
    adls_direct = ADLSConnector(env='prod', config_service=config_service)

Option 1: Add a convenience method to the factory 


In [ ]:
class DataStoreConnectionFactory:
    """
    Factory class for creating data store connections based on type
    """
    
    @staticmethod
    def create_connection(datastore_type: DataStoreType, env: str, 
                         config_service: Optional[ConfigurationService] = None,
                         auth_service: Optional[AuthenticationService] = None,
                         **kwargs):
        """
        Create a connection based on the datastore type
        
        Args:
            datastore_type (DataStoreType): Type of datastore to connect to
            env (str): Environment name
            config_service (ConfigurationService): Configuration service instance
            auth_service (AuthenticationService): Optional authentication service
            **kwargs: Additional arguments for specific connectors
            
        Returns:
            Connection object based on the datastore type
        """
        config_service = config_service or ConfigurationService()
        
        if datastore_type == DataStoreType.ADLS:
            return ADLSConnector(env=env, config_service=config_service, **kwargs)
        elif datastore_type == DataStoreType.KAFKA:
            return KafkaProducer(
                env=env, 
                config_service=config_service,
                auth_service=auth_service,
                **kwargs
            )
        else:
            raise ValueError(f"Unsupported datastore type: {datastore_type}")
    
    @staticmethod
    def create_adls_connection(env: str, storage_account: str, container: str, 
                              mount_name: str, subfolder: str = "", 
                              config_service: Optional[ConfigurationService] = None,
                              custom_secret_scope: Optional[str] = None,
                              safe_parent_path: str = "/mnt/temp",
                              auto_mount: bool = True) -> 'ADLSConnector':
        """
        Quick helper function to create and mount ADLS connection
        
        Args:
            env (str): Environment name
            storage_account (str): Azure storage account name
            container (str): Container name
            mount_name (str): Local mount name
            subfolder (str): Optional subfolder path
            config_service (ConfigurationService): Configuration service instance
            custom_secret_scope (str): Custom secret scope
            safe_parent_path (str): Safe parent path for operations
            auto_mount (bool): Whether to automatically mount after setup
            
        Returns:
            ADLSConnector: Configured and optionally mounted connector
        """
        connector = DataStoreConnectionFactory.create_connection(
            DataStoreType.ADLS,
            env=env,
            config_service=config_service,
            custom_secret_scope=custom_secret_scope,
            safe_parent_path=safe_parent_path
        )
        
        connector.setup_connection(storage_account, container, mount_name, subfolder)
        
        if auto_mount:
            connector.mount()
        
        return connector
    
    @staticmethod
    def create_kafka_producer(env: str, topic_name: Optional[str] = None,
                             config_service: Optional[ConfigurationService] = None,
                             auth_service: Optional[AuthenticationService] = None,
                             custom_secret_scope: Optional[str] = None) -> 'KafkaProducer':
        """
        Quick helper function to create Kafka producer
        
        Args:
            env (str): Environment name
            topic_name (str): Default topic name
            config_service (ConfigurationService): Configuration service instance
            auth_service (AuthenticationService): Authentication service
            custom_secret_scope (str): Custom secret scope
            
        Returns:
            KafkaProducer: Configured producer
        """
        return DataStoreConnectionFactory.create_connection(
            DataStoreType.KAFKA,
            env=env,
            topic_name=topic_name,
            config_service=config_service,
            auth_service=auth_service,
            custom_secret_scope=custom_secret_scope
        )

In [ ]:

# Usage with Option 1:
# Your familiar pattern - now with the factory!
adls = DataStoreConnectionFactory.create_adls_connection(
    env='nonprod',
    storage_account='myadls',
    container='analytics-data',
    mount_name='analytics_dev',
    custom_secret_scope='my-secrets'  # Note: parameter name changed
)

# Use it immediately
files = adls.list_files()

# Same for Kafka
kafka = DataStoreConnectionFactory.create_kafka_producer(
    env='prod',
    topic_name='my-topic'
)
kafka.send_message({"key": "value"})

authservice


In [ ]:
import time
import logging
from abc import ABC, abstractmethod
from typing import Dict, Any, Tuple, Optional
import requests


class AuthenticationService(ABC):
    """
    Abstract base class for authentication services
    """
    
    @abstractmethod
    def get_token(self) -> Tuple[str, float]:
        """
        Get authentication token
        
        Returns:
            Tuple[str, float]: (token, expiry_timestamp)
        """
        pass
    
    @abstractmethod
    def is_token_valid(self, expiry_timestamp: float) -> bool:
        """
        Check if token is still valid
        
        Args:
            expiry_timestamp (float): Token expiry timestamp
            
        Returns:
            bool: True if token is valid
        """
        pass


class OAuthAuthenticationService(AuthenticationService):
    """
    OAuth-based authentication service for getting tokens
    """
    
    def __init__(self, client_id: str, client_secret: str, token_url: str, 
                 scope: str = "kafka", grant_type: str = "client_credentials"):
        """
        Initialize OAuth authentication service
        
        Args:
            client_id (str): OAuth client ID
            client_secret (str): OAuth client secret
            token_url (str): Token endpoint URL
            scope (str): OAuth scope
            grant_type (str): OAuth grant type
        """
        self.client_id = client_id
        self.client_secret = client_secret
        self.token_url = token_url
        self.scope = scope
        self.grant_type = grant_type
        self.logger = logging.getLogger(__name__)
        
        # Token cache
        self._cached_token = None
        self._token_expiry = 0
    
    def get_token(self) -> Tuple[str, float]:
        """
        Get OAuth token, using cache if valid
        
        Returns:
            Tuple[str, float]: (access_token, expiry_timestamp)
        """
        # Return cached token if still valid
        if self._cached_token and self.is_token_valid(self._token_expiry):
            self.logger.debug("Using cached OAuth token")
            return self._cached_token, self._token_expiry
        
        # Get new token
        self.logger.info("Requesting new OAuth token")
        token, expiry = self._request_new_token()
        
        # Cache the token
        self._cached_token = token
        self._token_expiry = expiry
        
        return token, expiry
    
    def _request_new_token(self) -> Tuple[str, float]:
        """
        Request a new OAuth token from the server
        
        Returns:
            Tuple[str, float]: (access_token, expiry_timestamp)
        """
        payload = {
            'grant_type': self.grant_type,
            'scope': self.scope
        }
        
        try:
            response = requests.post(
                self.token_url,
                auth=(self.client_id, self.client_secret),
                data=payload,
                timeout=30  # Add timeout for safety
            )
            response.raise_for_status()  # Raise exception for HTTP errors
            
            token_data = response.json()
            access_token = token_data['access_token']
            expires_in = float(token_data['expires_in'])
            
            # Calculate expiry timestamp (with small buffer for safety)
            expiry_timestamp = time.time() + expires_in - 30  # 30 second buffer
            
            self.logger.info(f"Successfully obtained OAuth token, expires in {expires_in} seconds")
            return access_token, expiry_timestamp
            
        except requests.exceptions.RequestException as e:
            self.logger.error(f"Failed to obtain OAuth token: {str(e)}")
            raise Exception(f"OAuth token request failed: {str(e)}")
        except KeyError as e:
            self.logger.error(f"Invalid token response format: missing {str(e)}")
            raise Exception(f"Invalid token response: missing {str(e)}")
        except Exception as e:
            self.logger.error(f"Unexpected error during token request: {str(e)}")
            raise Exception(f"Token request error: {str(e)}")
    
    def is_token_valid(self, expiry_timestamp: float) -> bool:
        """
        Check if token is still valid
        
        Args:
            expiry_timestamp (float): Token expiry timestamp
            
        Returns:
            bool: True if token is valid
        """
        return time.time() < expiry_timestamp
    
    def invalidate_cache(self):
        """Invalidate cached token to force refresh on next request"""
        self._cached_token = None
        self._token_expiry = 0
        self.logger.info("OAuth token cache invalidated")


class AuthenticationServiceFactory:
    """
    Factory for creating authentication services
    """
    
    @staticmethod
    def create_oauth_service(client_id: str, client_secret: str, token_url: str, 
                           scope: str = "kafka") -> OAuthAuthenticationService:
        """
        Create OAuth authentication service
        
        Args:
            client_id (str): OAuth client ID
            client_secret (str): OAuth client secret
            token_url (str): Token endpoint URL
            scope (str): OAuth scope
            
        Returns:
            OAuthAuthenticationService: Configured OAuth service
        """
        return OAuthAuthenticationService(
            client_id=client_id,
            client_secret=client_secret,
            token_url=token_url,
            scope=scope
        )

new kafka producer

In [ ]:
import logging
import functools
import json
from typing import Dict, List, Optional, Union, Any
from confluent_kafka import Producer
from confluent_kafka.serialization import StringSerializer

# Import the new authentication service
from authentication_service import AuthenticationService, AuthenticationServiceFactory


class KafkaProducer:
    """
    Kafka producer for publishing messages to topics
    """
    
    def __init__(self, env: str = 'nonprod', topic_name: Optional[str] = None, 
                 config_service: Optional[ConfigurationService] = None, 
                 custom_secret_scope: Optional[str] = None,
                 auth_service: Optional[AuthenticationService] = None):
        """
        Initialize the Kafka producer
        
        Args:
            env (str): Environment ('prod' or 'nonprod')
            topic_name (str): Default topic name
            config_service (ConfigurationService): Configuration service instance
            custom_secret_scope (str): Custom secret scope
            auth_service (AuthenticationService): Optional authentication service
        """
        self.env = env
        self.topic_name = topic_name
        self.config_service = config_service or ConfigurationService()
        self.custom_secret_scope = custom_secret_scope
        self.logger = logging.getLogger(__name__)
        
        # Get resolved configuration (secrets resolved by ConfigurationService)
        self.env_config = self.config_service.get_resolved_configuration(
            env, 'kafka', custom_secret_scope
        )
        
        # Setup authentication service
        self.auth_service = auth_service or self._create_default_auth_service()
        
        # Setup other configuration
        self.bootstrap_servers = self.env_config['bootstrap-servers']
        self.ssl_ca_location = self.env_config['ssl-ca-location']
        
        # Create producer
        self.producer = Producer(self._get_producer_config())
        self.serializer = StringSerializer('utf8')
        
        self.logger.info(f"Kafka Producer initialized for {self.env} environment")
        self.logger.info(f"Bootstrap servers: {self.bootstrap_servers}")
    
    def _create_default_auth_service(self) -> AuthenticationService:
        """
        Create default OAuth authentication service from configuration
        
        Returns:
            AuthenticationService: Configured authentication service
        """
        return AuthenticationServiceFactory.create_oauth_service(
            client_id=self.env_config['client-id'],
            client_secret=self.env_config['client-secret'],
            token_url=self.env_config['token-url'],
            scope='kafka'
        )
    
    def _get_token_callback(self, config):
        """
        OAuth token callback for Kafka producer
        This method is called by the Kafka library when it needs a token
        
        Args:
            config: Kafka configuration (provided by confluent-kafka)
            
        Returns:
            Tuple: (token, expiry_timestamp)
        """
        try:
            token, expiry = self.auth_service.get_token()
            self.logger.debug("OAuth token provided to Kafka producer")
            return token, expiry
        except Exception as e:
            self.logger.error(f"Failed to get OAuth token for Kafka: {str(e)}")
            raise
    
    def _get_producer_config(self) -> Dict[str, Any]:
        """
        Get producer configuration with OAuth callback
        
        Returns:
            Dict[str, Any]: Kafka producer configuration
        """
        return {
            'bootstrap.servers': self.bootstrap_servers,
            'security.protocol': 'sasl_ssl',
            'sasl.mechanisms': 'OAUTHBEARER',
            'ssl.ca.location': self.ssl_ca_location,
            'oauth_cb': self._get_token_callback,  # Use our callback method
            'logger': self.logger,
        }
    
    def send_message(self, message: Union[Dict, str], topic_name: Optional[str] = None) -> bool:
        """
        Send a message to the specified topic
        
        Args:
            message (dict or str): Message to send
            topic_name (str): Topic to send to
            
        Returns:
            bool: Success status
        """
        if not topic_name and not self.topic_name:
            raise Exception("No topic specified. Provide topic_name parameter or set default topic.")
            
        target_topic = topic_name or self.topic_name
        
        # Convert message to JSON string if it's a dict
        if isinstance(message, dict):
            message = json.dumps(message)
        
        try:
            self.producer.produce(target_topic, value=message)
            self.producer.flush()
            self.logger.info(f"Message sent to topic '{target_topic}'")
            return True
        except Exception as e:
            self.logger.error(f"Failed to send message: {e}")
            return False
    
    def refresh_authentication(self):
        """
        Force refresh of authentication token
        Useful when token expires or authentication fails
        """
        if hasattr(self.auth_service, 'invalidate_cache'):
            self.auth_service.invalidate_cache()
            self.logger.info("Authentication cache invalidated")
    
    def close(self):
        """Close the producer connection"""
        self.producer.flush()
        self.logger.info("Producer closed")     

updated factory

In [ ]:
class DataStoreConnectionFactory:
    """
    Factory class for creating data store connections based on type
    """
    
    @staticmethod
    def create_connection(datastore_type: DataStoreType, env: str, 
                         config_service: Optional[ConfigurationService] = None,
                         auth_service: Optional[AuthenticationService] = None,
                         **kwargs):
        """
        Create a connection based on the datastore type
        
        Args:
            datastore_type (DataStoreType): Type of datastore to connect to
            env (str): Environment name
            config_service (ConfigurationService): Configuration service instance
            auth_service (AuthenticationService): Optional authentication service
            **kwargs: Additional arguments for specific connectors
            
        Returns:
            Connection object based on the datastore type
        """
        config_service = config_service or ConfigurationService()
        
        if datastore_type == DataStoreType.ADLS:
            return ADLSConnector(env=env, config_service=config_service, **kwargs)
        elif datastore_type == DataStoreType.KAFKA:
            return KafkaProducer(
                env=env, 
                config_service=config_service,
                auth_service=auth_service,  # Pass auth service to Kafka
                **kwargs
            )
        else:
            raise ValueError(f"Unsupported datastore type: {datastore_type}")

In [ ]:

# Example 1: Using default authentication (factory creates it)
kafka_producer = DataStoreConnectionFactory.create_connection(
    DataStoreType.KAFKA,
    env='prod',
    topic_name='my-topic'
)

# Example 2: Using custom authentication service
custom_auth = AuthenticationServiceFactory.create_oauth_service(
    client_id="my_client",
    client_secret="my_secret", 
    token_url="https://auth.company.com/token",
    scope="kafka"
)

kafka_producer = DataStoreConnectionFactory.create_connection(
    DataStoreType.KAFKA,
    env='prod',
    auth_service=custom_auth,
    topic_name='my-topic'
)

# Example 3: Force token refresh if needed
kafka_producer.refresh_authentication()

# Example 4: Direct authentication service usage
auth_service = AuthenticationServiceFactory.create_oauth_service(
    client_id="test", 
    client_secret="secret",
    token_url="https://auth.example.com"
)

token, expiry = auth_service.get_token()
print(f"Token: {token[:20]}..., Expires: {expiry}")